In [287]:
import urllib.request
from bs4 import BeautifulSoup
import pandas as pd
import requests
import folium
import numpy as np
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

In [3]:
url = 'https://crystalroof.co.uk/postcodes/SW12'
# open the url using urllib.request and put the HTML into the page variable
page = urllib.request.urlopen(url)
# parse the HTML from our URL into the BeautifulSoup parse tree format
soup = BeautifulSoup(page, "lxml")

In [4]:
# use the 'find_all' function to bring back all instances of the 'li' tag with class = postcodes__postcode
all_tables=soup.find_all('li', class_='postcodes__postcode')

In [5]:
postcodes = []
for row in all_tables:
    postcodes.append(row)

In [6]:
postcodes = [str(x) for x in postcodes]

In [7]:
postcodes = [x[76:] for x in postcodes]

In [8]:
postcodes = [x[:-9] for x in postcodes]

In [20]:
df = pd.DataFrame(postcodes,columns=["postCode"])
df.head()

,postCode
0,SW12 0AA
1,SW12 0AB
2,SW12 0AF
3,SW12 0AH
4,SW12 0AJ


In [29]:
def get_coordinates(api_key, address, verbose=False):
    try:
        url = 'https://maps.googleapis.com/maps/api/geocode/json?key={}&address={}'.format(api_key, address)
        response = requests.get(url).json()
        if verbose:
            print('Google Maps API JSON result =>', response)
        results = response['results']
        geographical_data = results[0]['geometry']['location'] # get geographical coordinates
        lat = geographical_data['lat']
        lon = geographical_data['lng']
        return [lat, lon]
    except:
        return [None, None]

In [31]:
api_key = 'your API key'
latitude = []
longitude = []
for x in df['postCode']:
    lat_lon = get_coordinates(api_key,x)
    latitude.append(lat_lon[0])
    longitude.append(lat_lon[1])    

In [35]:
df['latitude'] = latitude
df['longitude'] = longitude

In [259]:
df.head()

,postCode,latitude,longitude
0,SW12 0AA,NaN,NaN


In [37]:
df.to_csv('balham_lat-lon.csv')

In [261]:
df_csv = pd.read_csv('/Users/ciaran.brereton/Documents/data_science/data-science-capstone/balham_lat-lon.csv')
df_csv.head()

,postCode,latitude,longitude
0,SW12 0AA,51.448100,-0.142647
1,SW12 0AB,51.447770,-0.143745
2,SW12 0AF,51.447115,-0.142642
3,SW12 0AH,51.448185,-0.139805
4,SW12 0AJ,51.447860,-0.136869


In [159]:
CLIENT_ID = '05N1XEZQNIDGVQ2REY5LPG4R5QJL22VOJGGTQ5L4RK1EMJFU' # your Foursquare ID
CLIENT_SECRET = 'SEGUNZBYC1ABUB2OYNU1VAKVYU44R03BY3QIQSIUWS5XGQ4B' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
#LATITUDE = df['latitude'][0]
#LONGITUDE = df['longitude'][0]
#RADIUS = 150

In [262]:
def getNearbyVenues(names, latitudes, longitudes, radius=150):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}'.format(
                CLIENT_ID, 
                CLIENT_SECRET, 
                VERSION, 
                lat,
                lng,
                radius)

        # make the GET request
        results = requests.get(url).json()['response']['venues']
        for v in results:
            if v['categories'] != []:
                #print(v['categories'][0]['name'])
                #print(v['name'])
                #print(v['location']['lat'])
                #print(v['location']['lng'])
                venues_list.append([(name,lat,lng,
                                     v['name'],
                                     v['location']['lat'],
                                     v['location']['lng'],
                                     v['categories'][0]['name'])])
            else:
                continue 
        
        nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
        nearby_venues.columns = ['postCode', 
                  'Postcode Latitude', 
                  'Postcode Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)
    
        



In [249]:
#test_postcode = ['SW12 0AA']
#df_test = pd.DataFrame(test_postcode,columns=["postCode"])
#df_test['latitude'] = [51.448100]
#df_test['longitude'] = [-0.142647]
#df_test.head()

In [294]:
balham_venues = getNearbyVenues(names=df_csv['postCode'], latitudes=df_csv['latitude'], longitudes=df_csv['longitude'])

KeyboardInterrupt: 

In [295]:
balham_venues

,postCode,Postcode Latitude,Postcode Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,SW12 0AA,51.448100,-0.142647,La Retraite RC School,51.448123,-0.143380,High School
1,SW12 0AA,51.448100,-0.142647,St Bernadette's,51.447086,-0.143057,School
2,SW12 0AA,51.448100,-0.142647,Big Yellow Self Storage Company,51.446458,-0.144089,Storage Facility
3,SW12 0AA,51.448100,-0.142647,Pocklington Resource Centre,51.449378,-0.147123,Building
4,SW12 0AA,51.448100,-0.142647,BBC (Balham Banter Centre),51.449820,-0.144897,Comedy Club
...,...,...,...,...,...,...,...
15570,SW12 9ZS,51.446374,-0.151477,Hob,51.445194,-0.151075,Café
15571,SW12 9ZS,51.446374,-0.151477,Little Dessert Shop,51.445732,-0.150623,Dessert Shop
15572,SW12 9ZS,51.446374,-0.151477,Charlotte Cave Store & Services,51.445299,-0.150931,Health & Beauty Service
15573,SW12 9ZS,51.446374,-0.151477,Escape The Daily Grind,51.444892,-0.150731,Café


In [270]:
pd.set_option('display.max_rows', 500)
balham_venues.groupby('Venue Category').count()

,postCode,Postcode Latitude,Postcode Longitude,Venue,Venue Latitude,Venue Longitude
Venue Category,,,,,,
Antique Shop,1,1,1,1,1,1
Art Gallery,12,12,12,12,12,12
Arts & Crafts Store,28,28,28,28,28,28
Asian Restaurant,14,14,14,14,14,14
Athletics & Sports,8,8,8,8,8,8
Automotive Shop,40,40,40,40,40,40
BBQ Joint,24,24,24,24,24,24
Baby Store,60,60,60,60,60,60
Bakery,128,128,128,128,128,128


In [271]:
# one hot encoding
balham_onehot = pd.get_dummies(balham_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
balham_onehot['postCode'] = balham_venues['postCode'] 

# move neighborhood column to the first column
fixed_columns = [balham_onehot.columns[-1]] + list(balham_onehot.columns[:-1])
balham_onehot = balham_onehot[fixed_columns]

balham_onehot.head()

,postCode,Antique Shop,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Automotive Shop,BBQ Joint,Baby Store,Bakery,...,Vape Store,Vegetarian / Vegan Restaurant,Veterinarian,Vietnamese Restaurant,Warehouse,Weight Loss Center,Wine Bar,Wine Shop,Winery,Yoga Studio
0,SW12 0AA,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,SW12 0AA,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,SW12 0AA,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,SW12 0AA,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,SW12 0AA,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [272]:
balham_onehot.shape

(15575, 201)

In [273]:
balham_grouped = balham_onehot.groupby('postCode').mean().reset_index()
balham_grouped

,postCode,Antique Shop,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Automotive Shop,BBQ Joint,Baby Store,Bakery,...,Vape Store,Vegetarian / Vegan Restaurant,Veterinarian,Vietnamese Restaurant,Warehouse,Weight Loss Center,Wine Bar,Wine Shop,Winery,Yoga Studio
0,SW12 0AA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000
1,SW12 0AB,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000
2,SW12 0AF,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000
3,SW12 0AH,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.0,0.000000,0.0,0.000000,0.038462,0.0,0.0,0.0,0.0,0.000000
4,SW12 0AJ,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.0,0.000000,0.0,0.000000,0.037037,0.0,0.0,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
591,SW12 9SY,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.0,0.000000,0.0,0.033333,0.000000,0.0,0.0,0.0,0.0,0.033333
592,SW12 9TT,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.033333,0.033333,...,0.0,0.033333,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000
593,SW12 9YJ,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.033333,0.000000,...,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000
594,SW12 9ZL,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.033333,0.000000,...,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000


In [274]:
num_top_venues = 5

for code in balham_grouped['postCode']:
    print("----"+code+"----")
    temp = balham_grouped[balham_grouped['postCode'] == code].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----SW12 0AA----
            venue  freq
0          Office  0.25
1        Building  0.17
2  Hardware Store  0.08
3          School  0.08
4    Meeting Room  0.04


----SW12 0AB----
            venue  freq
0          Office  0.12
1        Building  0.12
2  Hardware Store  0.12
3             Pub  0.08
4     High School  0.04


----SW12 0AF----
            venue  freq
0          Office  0.17
1        Building  0.13
2  Hardware Store  0.13
3     Event Space  0.04
4         Daycare  0.04


----SW12 0AH----
        venue  freq
0    Building  0.12
1    Bus Stop  0.12
2  Playground  0.08
3      School  0.08
4      Church  0.04


----SW12 0AJ----
           venue  freq
0       Bus Stop  0.11
1       Building  0.11
2         School  0.07
3  Grocery Store  0.07
4           Park  0.07


----SW12 0AL----
        venue  freq
0    Building  0.17
1        Park  0.12
2  Playground  0.08
3    Bus Stop  0.08
4      School  0.08


----SW12 0AN----
           venue  freq
0       Building  0.15
1         Sch

                 venue  freq
0          Coffee Shop  0.10
1             Building  0.10
2        Grocery Store  0.05
3              Daycare  0.05
4  Government Building  0.05


----SW12 0HS----
            venue  freq
0    Tennis Court  0.11
1     Coffee Shop  0.11
2        Building  0.11
3     Gas Station  0.05
4  Cosmetics Shop  0.05


----SW12 0HW----
                 venue  freq
0               Office  0.21
1             Building  0.11
2  Government Building  0.05
3         Trade School  0.05
4              Daycare  0.05


----SW12 0HY----
              venue  freq
0    Hardware Store  0.11
1            School  0.07
2   Doctor's Office  0.04
3  Storage Facility  0.04
4              Café  0.04


----SW12 0HZ----
               venue  freq
0             School  0.12
1             Office  0.12
2     Hardware Store  0.08
3    Doctor's Office  0.04
4  Mobile Phone Shop  0.04


----SW12 0JA----
             venue  freq
0           School  0.10
1   Hardware Store  0.10
2      Coffee Shop  

4                 Spa  0.03


----SW12 8AA----
                venue  freq
0  Salon / Barbershop  0.14
1         Coffee Shop  0.07
2                 Pub  0.07
3   Indian Restaurant  0.07
4              School  0.03


----SW12 8AB----
                       venue  freq
0         Salon / Barbershop  0.11
1                        Pub  0.07
2                   Bus Stop  0.07
3  College Academic Building  0.04
4             Nursery School  0.04


----SW12 8AD----
                  venue  freq
0         Grocery Store  0.15
1  Fast Food Restaurant  0.08
2              Bus Stop  0.08
3    Real Estate Office  0.04
4          Funeral Home  0.04


----SW12 8AE----
                  venue  freq
0         Grocery Store  0.16
1  Fast Food Restaurant  0.08
2         Metro Station  0.04
3                Church  0.04
4        Cosmetics Shop  0.04


----SW12 8AF----
                  venue  freq
0         Grocery Store  0.12
1              Bus Stop  0.12
2          Tech Startup  0.08
3  Caucasian Restau

4    Train Station  0.04


----SW12 8HS----
             venue  freq
0  Doctor's Office  0.10
1              Pub  0.10
2    Grocery Store  0.10
3     Tech Startup  0.10
4       Beer Store  0.05


----SW12 8HT----
             venue  freq
0  Doctor's Office  0.09
1       Beer Store  0.05
2      Coffee Shop  0.05
3       Non-Profit  0.05
4    Grocery Store  0.05


----SW12 8JA----
                venue  freq
0         Coffee Shop  0.10
1  Salon / Barbershop  0.07
2      Sandwich Place  0.07
3            Platform  0.07
4                Café  0.07


----SW12 8JB----
         venue  freq
0  Coffee Shop  0.10
1          Pub  0.07
2       Church  0.07
3         Café  0.07
4       Bakery  0.07


----SW12 8JD----
            venue  freq
0             Pub  0.10
1     Coffee Shop  0.10
2            Café  0.07
3  Sandwich Place  0.07
4     Supermarket  0.07


----SW12 8JE----
               venue  freq
0           Building  0.13
1             Winery  0.09
2  Elementary School  0.09
3             C

4           Church  0.09


----SW12 8QN----
            venue  freq
0          Church  0.10
1        Building  0.10
2         Library  0.05
3  Baseball Field  0.05
4     Coffee Shop  0.05


----SW12 8QP----
                venue  freq
0         Post Office  0.07
1         Coffee Shop  0.07
2                 Pub  0.07
3   Indian Restaurant  0.07
4  Salon / Barbershop  0.07


----SW12 8QQ----
              venue  freq
0     Grocery Store  0.07
1            Church  0.07
2          Building  0.07
3  Community Center  0.03
4       Supermarket  0.03


----SW12 8QR----
                venue  freq
0  Salon / Barbershop  0.07
1              Bakery  0.07
2    Dentist's Office  0.07
3                Bank  0.07
4                 Pub  0.07


----SW12 8QT----
            venue  freq
0   Grocery Store  0.12
1          Church  0.08
2     Dry Cleaner  0.04
3        Building  0.04
4  Nursery School  0.04


----SW12 8QW----
                venue  freq
0         Post Office  0.07
1                 Pub  0.

                venue  freq
0         Coffee Shop  0.10
1         Post Office  0.07
2                 Pub  0.07
3         Supermarket  0.07
4  Salon / Barbershop  0.07


----SW12 9AR----
                venue  freq
0  Salon / Barbershop  0.13
1         Coffee Shop  0.10
2              Bakery  0.07
3    Dentist's Office  0.07
4         Supermarket  0.07


----SW12 9AS----
                venue  freq
0  Salon / Barbershop  0.13
1         Coffee Shop  0.10
2              Bakery  0.07
3      Sandwich Place  0.03
4    Stationery Store  0.03


----SW12 9AT----
                venue  freq
0         Coffee Shop  0.13
1  Salon / Barbershop  0.10
2      Cosmetics Shop  0.07
3              Bakery  0.07
4         Supermarket  0.07


----SW12 9AU----
            venue  freq
0     Coffee Shop  0.10
1     Supermarket  0.07
2  Sandwich Place  0.07
3          Bakery  0.07
4  Cosmetics Shop  0.07


----SW12 9AW----
                  venue  freq
0           Coffee Shop  0.07
1     Indian Restaurant  0.07

           venue  freq
0  Grocery Store  0.12
1   Tech Startup  0.08
2       Bus Stop  0.08
3       Building  0.04
4    Supermarket  0.04


----SW12 9LH----
                  venue  freq
0         Grocery Store  0.12
1  Fast Food Restaurant  0.08
2    Salon / Barbershop  0.08
3              Bus Stop  0.08
4     French Restaurant  0.04


----SW12 9LJ----
                  venue  freq
0         Grocery Store  0.11
1     Indian Restaurant  0.07
2    Salon / Barbershop  0.07
3  Fast Food Restaurant  0.07
4        Nursery School  0.04


----SW12 9LL----
                       venue  freq
0              Grocery Store  0.12
1       Fast Food Restaurant  0.08
2                        Pub  0.04
3             Breakfast Spot  0.04
4  College Academic Building  0.04


----SW12 9LP----
             venue  freq
0  Doctor's Office  0.07
1      Coffee Shop  0.07
2       Playground  0.07
3   Sandwich Place  0.04
4        Pet Store  0.04


----SW12 9LQ----
                  venue  freq
0         Grocery

4           Bike Shop  0.03


----SW12 9SG----
               venue  freq
0           Platform  0.13
1        Coffee Shop  0.10
2               Café  0.07
3                Bar  0.07
4  Indian Restaurant  0.07


----SW12 9SH----
           venue  freq
0       Platform  0.13
1    Coffee Shop  0.10
2            Bar  0.07
3  Grocery Store  0.03
4         Office  0.03


----SW12 9SJ----
                  venue  freq
0           Coffee Shop  0.13
1              Platform  0.13
2                   Pub  0.07
3                   Bar  0.07
4  Fast Food Restaurant  0.03


----SW12 9SL----
                 venue  freq
0          Coffee Shop  0.13
1             Platform  0.13
2          Supermarket  0.07
3    Indian Restaurant  0.07
4  Moroccan Restaurant  0.03


----SW12 9SN----
               venue  freq
0           Platform  0.10
1        Coffee Shop  0.10
2  Indian Restaurant  0.07
3        Yoga Studio  0.03
4        Pizza Place  0.03


----SW12 9SR----
                  venue  freq
0         Gr

In [275]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [278]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['postCode']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
postCodes_venues_sorted = pd.DataFrame(columns=columns)
postCodes_venues_sorted['postCode'] = balham_grouped['postCode']

for ind in np.arange(balham_grouped.shape[0]):
    postCodes_venues_sorted.iloc[ind, 1:] = return_most_common_venues(balham_grouped.iloc[ind, :], num_top_venues)

postCodes_venues_sorted.head()

,postCode,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,SW12 0AA,Office,Building,School,Hardware Store,Italian Restaurant,Grocery Store,Pet Service,Convenience Store,Comedy Club,Storage Facility
1,SW12 0AB,Office,Hardware Store,Building,Pub,Italian Restaurant,Medical Center,Event Space,Pet Service,Recruiting Agency,Convenience Store
2,SW12 0AF,Office,Hardware Store,Building,Italian Restaurant,Tech Startup,High School,Grocery Store,Meeting Room,Pet Service,Event Space
3,SW12 0AH,Bus Stop,Building,Playground,School,Grocery Store,Daycare,Elementary School,Nursery School,Church,French Restaurant
4,SW12 0AJ,Bus Stop,Building,Grocery Store,Park,School,Playground,Elementary School,Warehouse,Storage Facility,Distribution Center


In [280]:
# set number of clusters
kclusters = 5

balham_grouped_clustering = balham_grouped.drop('postCode', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(balham_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 2, 2, 1, 1, 1, 1, 1, 2, 1], dtype=int32)

In [283]:
# add clustering labels
postCodes_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

ValueError: cannot insert Cluster Labels, already exists

In [284]:
balham_merged = df_csv

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
balham_merged = balham_merged.join(postCodes_venues_sorted.set_index('postCode'), on='postCode')

balham_merged.head() # check the last columns!

,postCode,latitude,longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,SW12 0AA,51.448100,-0.142647,2,Office,Building,School,Hardware Store,Italian Restaurant,Grocery Store,Pet Service,Convenience Store,Comedy Club,Storage Facility
1,SW12 0AB,51.447770,-0.143745,2,Office,Hardware Store,Building,Pub,Italian Restaurant,Medical Center,Event Space,Pet Service,Recruiting Agency,Convenience Store
2,SW12 0AF,51.447115,-0.142642,2,Office,Hardware Store,Building,Italian Restaurant,Tech Startup,High School,Grocery Store,Meeting Room,Pet Service,Event Space
3,SW12 0AH,51.448185,-0.139805,1,Bus Stop,Building,Playground,School,Grocery Store,Daycare,Elementary School,Nursery School,Church,French Restaurant
4,SW12 0AJ,51.447860,-0.136869,1,Bus Stop,Building,Grocery Store,Park,School,Playground,Elementary School,Warehouse,Storage Facility,Distribution Center


In [288]:
# create map
latitude = 51.4440
longitude = 0.1494
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(balham_merged['latitude'], balham_merged['longitude'], balham_merged['postCode'], balham_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [300]:
balham_merged.loc[balham_merged['Cluster Labels'] == 0, balham_merged.columns[[1] + list(range(4, balham_merged.shape[1]))]]


,latitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
54,51.443513,Church,Office,Gas Station,Coworking Space,Food Truck,Storage Facility,Building,Coffee Shop,Tennis Court,Government Building
78,51.439736,Building,Office,Hotel,Government Building,Doctor's Office,Elementary School,Park,Nursery School,Coffee Shop,School
81,51.441543,Office,Coffee Shop,Building,Gas Station,Tennis Court,Elementary School,Park,Daycare,Cosmetics Shop,Non-Profit
192,51.450053,Pub,Real Estate Office,Office,Grocery Store,Tech Startup,Church,Italian Restaurant,Doctor's Office,Building,Bus Stop
204,51.443487,Café,Bar,School,College Theater,Bus Stop,French Restaurant,Church,Coffee Shop,Residential Building (Apartment / Condo),Tech Startup
207,51.445375,Café,Office,Church,Building,Community Center,Train Station,Non-Profit,School,Salon / Barbershop,Tech Startup
208,51.445538,Church,Building,French Restaurant,Train Station,Garden,Storage Facility,School,Non-Profit,Salon / Barbershop,College Theater
209,51.445043,Office,Building,Pub,Coffee Shop,Café,Brewery,Elementary School,Fast Food Restaurant,Record Shop,Cosmetics Shop
211,51.444921,Building,Coffee Shop,Office,Pub,Doctor's Office,Record Shop,Mobile Phone Shop,Platform,Café,Non-Profit
213,51.443240,School,Residential Building (Apartment / Condo),Ice Cream Shop,Grocery Store,Military Base,Doctor's Office,Bus Stop,College Theater,Storage Facility,Salon / Barbershop


In [301]:
balham_merged.loc[balham_merged['Cluster Labels'] == 1, balham_merged.columns[[1] + list(range(4, balham_merged.shape[1]))]]


,latitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,51.448185,Bus Stop,Building,Playground,School,Grocery Store,Daycare,Elementary School,Nursery School,Church,French Restaurant
4,51.447860,Bus Stop,Building,Grocery Store,Park,School,Playground,Elementary School,Warehouse,Storage Facility,Distribution Center
5,51.448190,Building,Park,Playground,School,Bus Stop,Housing Development,Gym / Fitness Center,Storage Facility,Coffee Shop,Shipping Store
6,51.447862,Building,Playground,Grocery Store,Park,School,Bus Stop,Elementary School,Daycare,Coffee Shop,Shipping Store
7,51.447703,Bus Stop,Grocery Store,Building,Park,Fast Food Restaurant,Winery,Shipping Store,Housing Development,Food Truck,Pub
9,51.447454,Bus Stop,Building,Grocery Store,Coffee Shop,Park,Shipping Store,Gym / Fitness Center,Coworking Space,Fast Food Restaurant,Daycare
10,51.447456,Bus Stop,Grocery Store,Building,Pub,Shipping Store,Fast Food Restaurant,Gym / Fitness Center,Coworking Space,Daycare,Food Truck
11,51.446553,Bus Stop,Building,Grocery Store,Park,Coffee Shop,Gym / Fitness Center,Fast Food Restaurant,Daycare,Coworking Space,Food Truck
12,51.445815,Bus Stop,Grocery Store,Building,Park,Coworking Space,Bus Line,Gym / Fitness Center,Coffee Shop,Daycare,Fast Food Restaurant
13,51.447453,Bus Stop,Building,Grocery Store,Park,Daycare,Church,Shipping Store,Coffee Shop,School,Gym / Fitness Center


In [302]:
balham_merged.loc[balham_merged['Cluster Labels'] == 2, balham_merged.columns[[1] + list(range(4, balham_merged.shape[1]))]]


,latitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,51.448100,Office,Building,School,Hardware Store,Italian Restaurant,Grocery Store,Pet Service,Convenience Store,Comedy Club,Storage Facility
1,51.447770,Office,Hardware Store,Building,Pub,Italian Restaurant,Medical Center,Event Space,Pet Service,Recruiting Agency,Convenience Store
2,51.447115,Office,Hardware Store,Building,Italian Restaurant,Tech Startup,High School,Grocery Store,Meeting Room,Pet Service,Event Space
8,51.448429,Building,Office,School,Hardware Store,Housing Development,Tech Startup,Professional & Other Places,Coworking Space,Comedy Club,Nursery School
20,51.443750,Office,Hardware Store,School,Daycare,Meeting Room,Café,Doctor's Office,Spa,Mobile Phone Shop,Moroccan Restaurant
25,51.446866,School,Pub,Building,Pet Service,Grocery Store,Scenic Lookout,Professional & Other Places,Nursery School,Meeting Room,High School
26,51.447856,Building,Office,School,Playground,Church,Hardware Store,Industrial Estate,Tech Startup,Daycare,Professional & Other Places
27,51.445471,Hardware Store,School,Bus Line,Pet Service,Pub,Italian Restaurant,Distillery,Spa,Flower Shop,Music Store
28,51.446701,School,Pub,Building,Bus Line,Pet Service,Hardware Store,Scenic Lookout,Professional & Other Places,Plaza,Nursery School
29,51.447522,School,Building,Coffee Shop,Pet Service,Hardware Store,Scenic Lookout,Salon / Barbershop,Pub,Professional & Other Places,Nursery School


In [303]:
balham_merged.loc[balham_merged['Cluster Labels'] == 3, balham_merged.columns[[1] + list(range(4, balham_merged.shape[1]))]]


,latitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
23,51.452777,Coffee Shop,Restaurant,Grocery Store,Supermarket,Park,Pub,Farmers Market,Police Station,Salon / Barbershop,Tech Startup
114,51.443434,Bus Line,Coffee Shop,Post Office,Café,Coworking Space,Gas Station,Bus Stop,Food Truck,Building,Comedy Club
158,51.443989,Coffee Shop,Salon / Barbershop,Platform,Spa,Burger Joint,Fish & Chips Shop,Supermarket,Convenience Store,Market,Pub
159,51.446374,Coffee Shop,Dentist's Office,Café,Bank,Post Office,Italian Restaurant,Government Building,Funeral Home,Furniture / Home Store,Spa
160,51.446374,Coffee Shop,Dentist's Office,Café,Bank,Post Office,Italian Restaurant,Government Building,Funeral Home,Furniture / Home Store,Spa
161,51.446374,Coffee Shop,Dentist's Office,Café,Bank,Post Office,Italian Restaurant,Government Building,Funeral Home,Furniture / Home Store,Spa
162,51.446374,Coffee Shop,Dentist's Office,Café,Bank,Post Office,Italian Restaurant,Government Building,Funeral Home,Furniture / Home Store,Spa
163,51.446374,Coffee Shop,Dentist's Office,Café,Bank,Post Office,Italian Restaurant,Government Building,Funeral Home,Furniture / Home Store,Spa
164,51.443989,Coffee Shop,Salon / Barbershop,Platform,Spa,Burger Joint,Fish & Chips Shop,Supermarket,Convenience Store,Market,Pub
165,51.443989,Coffee Shop,Salon / Barbershop,Platform,Spa,Burger Joint,Fish & Chips Shop,Supermarket,Convenience Store,Market,Pub


In [304]:
balham_merged.loc[balham_merged['Cluster Labels'] == 4, balham_merged.columns[[1] + list(range(4, balham_merged.shape[1]))]]


,latitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
15,51.444502,Grocery Store,Bus Stop,Bus Line,Post Office,Café,Salon / Barbershop,Residential Building (Apartment / Condo),Fast Food Restaurant,Park,Gym / Fitness Center
22,51.453432,Grocery Store,Coffee Shop,Supermarket,Deli / Bodega,Salon / Barbershop,Bus Stop,Mexican Restaurant,Café,Fishing Spot,Restaurant
24,51.449741,Bus Stop,Grocery Store,School,Building,Tech Startup,Gas Station,Gastropub,Storage Facility,Supermarket,Coffee Shop
33,51.449082,Bus Stop,Hardware Store,Grocery Store,School,Tech Startup,Gastropub,Gas Station,Distribution Center,French Restaurant,Nursery School
34,51.449493,Grocery Store,Bus Stop,Building,Tech Startup,Park,French Restaurant,School,Gas Station,Nursery School,Storage Facility
35,51.450150,Grocery Store,Bus Stop,Tech Startup,School,Bed & Breakfast,Supermarket,Salon / Barbershop,Office,Gas Station,Real Estate Office
36,51.450643,Bus Stop,Tech Startup,School,Grocery Store,Fish & Chips Shop,Garden,Gas Station,Gastropub,Metro Station,Coffee Shop
37,51.451464,Supermarket,Bus Stop,Grocery Store,Garden,School,Gas Station,Caucasian Restaurant,Coffee Shop,Furniture / Home Store,Tech Startup
38,51.452366,Supermarket,Grocery Store,Yoga Studio,Garden,Bus Stop,Café,Spa,Mexican Restaurant,Coffee Shop,Nail Salon
39,51.453430,Supermarket,Grocery Store,Café,Chinese Restaurant,Indian Restaurant,Metro Station,Mexican Restaurant,Bus Stop,Fried Chicken Joint,Coffee Shop
